![Piggy bank](piggy_bank.jpg)

Personal loans are a lucrative revenue stream for banks. The typical interest rate of a two-year loan in the United Kingdom is [around 10%](https://www.experian.com/blogs/ask-experian/whats-a-good-interest-rate-for-a-personal-loan/). This might not sound like a lot, but in September 2022 alone UK consumers borrowed [around £1.5 billion](https://www.ukfinance.org.uk/system/files/2022-12/Household%20Finance%20Review%202022%20Q3-%20Final.pdf), which would mean approximately £300 million in interest generated by banks over two years!

You have been asked to work with a bank to clean the data they collected as part of a recent marketing campaign, which aimed to get customers to take out a personal loan. They plan to conduct more marketing campaigns going forward so would like you to ensure it conforms to the specific structure and data types that they specify so that they can then use the cleaned data you provide to set up a PostgreSQL database, which will store this campaign's data and allow data from future campaigns to be easily imported.

They have supplied you with a csv file called `"bank_marketing.csv"`, which you will need to clean, reformat, and split the data, saving three final csv files. Specifically, the three files should have the names and contents as outlined below:

## `client.csv`

| column           | data type | description                                                | cleaning requirements                             |
| ---------------- | --------- | ---------------------------------------------------------- | ------------------------------------------------- |
| `client_id`      | `integer` | Client ID                                                  | N/A                                               |
| `age`            | `integer` | Client's age in years                                      | N/A                                               |
| `job`            | `object`  | Client's type of job                                       | Change `"."` to `"_"`                             |
| `marital`        | `object`  | Client's marital status                                    | N/A                                               |
| `education`      | `object`  | Client's level of education                                | Change `"."` to `"_"` and `"unknown"` to `np.NaN` |
| `credit_default` | `bool`    | Whether the client's credit is in default                  | Convert to boolean data type                      |
| `mortgage`       | `bool`    | Whether the client has an existing mortgage (housing loan) | Convert to boolean data type                      |

<br>

## `campaign.csv`

| column                       | data type  | description                                                       | cleaning requirements                                                                                                                                  |
| ---------------------------- | ---------- | ----------------------------------------------------------------- | ------------------------------------------------------------------------------------------------------------------------------------------------------ |
| `client_id`                  | `integer`  | Client ID                                                         | N/A                                                                                                                                                    |
| `number_contacts`            | `integer`  | Number of contact attempts to the client in the current campaign  | N/A                                                                                                                                                    |
| `contact_duration`           | `integer`  | Last contact duration in seconds                                  | N/A                                                                                                                                                    |
| `previous_campaign_contacts` | `integer`  | Number of contact attempts to the client in the previous campaign | N/A                                                                                                                                                    |
| `previous_outcome`           | `bool`     | Outcome of the previous campaign                                  | Convert to boolean data type                                                                                                                           |
| `campaign_outcome`           | `bool`     | Outcome of the current campaign                                   | Convert to boolean data type                                                                                                                           |
| `last_contact_date`          | `datetime` | Last date the client was contacted                                | Create from a combination of `day`, `month`, and a newly created `year` column (which should have a value of `2022`); <br> **Format =** `"YYYY-MM-DD"` |

<br>

## `economics.csv`

| column                 | data type | description                                                              | cleaning requirements |
| ---------------------- | --------- | ------------------------------------------------------------------------ | --------------------- |
| `client_id`            | `integer` | Client ID                                                                | N/A                   |
| `cons_price_idx`       | `float`   | Consumer price index (monthly indicator)                                 | N/A                   |
| `euribor_three_months` | `float`   | Euro Interbank Offered Rate (euribor) three-month rate (daily indicator) | N/A                   |


In [13]:
import pandas as pd
import numpy as np

bank_marketing_df = pd.read_csv("bank_marketing.csv")

bank_marketing_df.info()

client_cols = [
    "client_id",
    "age",
    "job",
    "marital",
    "education",
    "credit_default",
    "mortgage",
]
campaign_cols = [
    "client_id",
    "number_contacts",
    "contact_duration",
    "previous_campaign_contacts",
    "previous_outcome",
    "campaign_outcome",
    "month",
    "day",
]
economics_cols = ["client_id", "cons_price_idx", "euribor_three_months"]

client = bank_marketing_df[client_cols]
client["job"] = client["job"].str.replace(".", "_")
client["education"] = client["education"].str.replace(".", "_")
client["education"] = client["education"].replace("unknown", np.nan)
client = client.astype({"credit_default": "bool", "mortgage": "bool"})

campaign = bank_marketing_df[campaign_cols]
campaign = campaign.astype(
    {"previous_outcome": "bool", "campaign_outcome": "bool"}
)
month_mapping = {
    "jan": 1,
    "feb": 2,
    "mar": 3,
    "apr": 4,
    "may": 5,
    "jun": 6,
    "jul": 7,
    "aug": 8,
    "sep": 9,
    "oct": 10,
    "nov": 11,
    "dec": 12,
}
campaign["month"] = campaign["month"].str.lower().map(month_mapping)

# Create last_contact_date column
campaign["year"] = 2022
campaign["last_contact_date"] = pd.to_datetime(campaign[["year", "month", "day"]])
campaign = campaign.drop(["year", "month", "day"], axis=1)

economics = bank_marketing_df[economics_cols]

client.to_csv("client.csv", index=False)

campaign.to_csv("campaign.csv", index=False)

economics.to_csv("economics.csv", index=False)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41188 entries, 0 to 41187
Data columns (total 16 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   client_id                   41188 non-null  int64  
 1   age                         41188 non-null  int64  
 2   job                         41188 non-null  object 
 3   marital                     41188 non-null  object 
 4   education                   41188 non-null  object 
 5   credit_default              41188 non-null  object 
 6   mortgage                    41188 non-null  object 
 7   month                       41188 non-null  object 
 8   day                         41188 non-null  int64  
 9   contact_duration            41188 non-null  int64  
 10  number_contacts             41188 non-null  int64  
 11  previous_campaign_contacts  41188 non-null  int64  
 12  previous_outcome            41188 non-null  object 
 13  cons_price_idx              411

C:\Users\dpl2k\AppData\Local\Temp\ipykernel_22444\4281696063.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  client["job"] = client["job"].str.replace(".", "_")
C:\Users\dpl2k\AppData\Local\Temp\ipykernel_22444\4281696063.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  client["education"] = client["education"].str.replace(".", "_")
C:\Users\dpl2k\AppData\Local\Temp\ipykernel_22444\4281696063.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try usin